In [1]:
clean_up = True
%run stdPackages.ipynb

The file _gams_py_gdb0.gdx is still active and was not deleted.


In [2]:
name = 'A'
ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])
with open(f"{d['data']}\\glob_{name}","rb") as file: # load global settings anmed glob_name
    glob=pickle.load(file)

In [3]:
def loadFromPickle(x):
    with open(os.path.join(d['data'],x), "rb") as file:
        return pickle.load(file)

### Test some methods

### 1: Load a module and re-run:

*load pickle:*

In [4]:
module = name+'Pi'
f = os.path.join(d['data'], module)
P = CGE_Production.Production_ExoMu(f = f)

*Note: It might be a good idea to use the ```write``` method again, if any of the local placeholders have changed names (e.g. if we have added a new placeholder):*

In [5]:
P.write();
m = P.run(exportTo = d['work'],ws=ws,**{'cns': 'CONOPT4'}) # solve the model using CONOPT4.

### 2: Combine modules

Standardized approach to combining multiple modules:
1. Re-compile settings, re-write text, and re-run. Using this method, the only thing we do by loading the modules - instead of initializing them from scratch - is to use the databases.
2. Merge ```GmsSettings``` objects when loading: Doing this, we should be vary that a couple of things might cause issues here: (i) The gams code may refer to databases with other names (%name%) syntax. (ii) The standard arguments can be repeated for all the modules we load.

In [6]:
modules = [name+k for k in ('Pi','Ii','_itory','HH','Trade')]
P = CGE_GmsPython.GmsPython(name = name, glob=glob, m = [loadFromPickle(n) for n in modules], s_kwargs = {'ws': ws})
P.compile() # re-compile settings
P.write() # re-write text
m = P.run(exportTo = d['work'],ws=ws,**{'cns': 'CONOPT4'}) # solve the model using CONOPT4.

*Change to calibration state, if needed:*

In [7]:
P.s.setstate('C')
P.write();